In [1]:
!pip install transformers datasets evaluate peft trl bitsandbytes

import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer

base_model = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
new_model = "llama-1.1B-rare_disease"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# -*- coding: utf-8 -*-
model = AutoModelForCausalLM.from_pretrained(base_model, device_map='auto')
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # pad sequences
tokenizer.padding_side = 'right'

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [6]:
# run inference
logging.set_verbosity(logging.CRITICAL)
prompt = "is cystic fibrosis a rare diease?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] is cystic fibrosis a rare diease? [/INST]
[INST] is cystic fibrosis a rare diease? [/INST]
[INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cystic fibrosis a rare diease? [/INST] [INST] is cyst


In [33]:
import pandas as pd
from datasets import load_dataset, Dataset

data_=pd.read_csv('/content/FT_data_v2.csv')
def transform(x, y, z):
    if z == 'both description':
        return f"<s>[INST] What does this concept mean? {x} [/INST] {y} </s>"
    elif z == 'description':
        return f"<s>[INST] Describe {x}: [/INST] {x} is a rare disease. {y} </s>"
    elif z == 'synonym':
        return f"<s>[INST] What is the synonym of {x}? [/INST] {y} is a synonym of {x}. </s>"
    else:
        return f"<s>[INST] Explain {x}: [/INST] {y} </s>"  # Default case

data_['text']= data_.apply(lambda x: transform(x['sentence1'], x['sentence2'], x['similarity_score']),axis=1)
#data_=data_.drop(['Unnamed: 0', 'sentence1', 'sentence2', 'similarity_score'],axis=1)
#data_.to_csv('data_.csv')
#dataset = load_dataset("csv", data_files="/content/data_.csv")
#data_

# Keep only the required 'text' column
data_ = data_[['text']]
# Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(data_)
# Save dataset to disk (optional)
dataset.save_to_disk("formatted_dataset")
# Verify dataset structure
print(dataset)

Saving the dataset (0/1 shards):   0%|          | 0/38110 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 38110
})


In [25]:
dataset = load_dataset(guanaco_dataset, split="train")

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [39]:
peft_params = LoraConfig(lora_alpha=16, # multiplier of Lora output when its added to the full forward output
                         lora_dropout=0.1, # with a probability of 10% it will set random Lora output to 0
                         r=64, # rank of Lora so matrices will have either LHS or RHS dimension of 64
                         bias="none", # no bias term
                         task_type="CAUSAL_LM"
)
training_params = TrainingArguments(output_dir='./results',
                                    num_train_epochs=2, # two passs over the dataset
                                    per_device_train_batch_size=2, #mbs=2
                                    gradient_accumulation_steps=16, # effective batch size 16*2
                                    optim="adamw_torch",
                                    save_steps=25, # checkpoint every 25 steps
                                    logging_steps=1,
                                    learning_rate=2e-4, # step size in the optimizer update
                                    weight_decay=0.001,
                                    fp16=True, # 16 bit
                                    bf16=False, # not supported on V100
                                    max_grad_norm=0.3, #gradient clipping improves convergence
                                    max_steps=-1,
                                    warmup_ratio=0.03, # learning rate warmup
                                    group_by_length=True,
                                    lr_scheduler_type="cosine" # cosine lr scheduler
)

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    peft_config=peft_params,  # Parameter-efficient fine-tuning (LoRA)
    args=training_params
)


import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() # clean cache

trainer.train() # train the model
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

Converting train dataset to ChatML:   0%|          | 0/38110 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/38110 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/38110 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/38110 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jabervalinejad (jabervalinejad-virginia-tech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'loss': 1.7489, 'grad_norm': 0.08775021880865097, 'learning_rate': 2.777777777777778e-06, 'mean_token_accuracy': 0.6023471690714359, 'epoch': 0.0008396746260823931}
{'loss': 1.954, 'grad_norm': 0.11715579032897949, 'learning_rate': 5.555555555555556e-06, 'mean_token_accuracy': 0.5940355099737644, 'epoch': 0.0016793492521647861}
{'loss': 2.1154, 'grad_norm': 0.13042525947093964, 'learning_rate': 8.333333333333334e-06, 'mean_token_accuracy': 0.5731521584093571, 'epoch': 0.0025190238782471794}
{'loss': 2.1514, 'grad_norm': 0.16400210559368134, 'learning_rate': 1.1111111111111112e-05, 'mean_token_accuracy': 0.5730199255049229, 'epoch': 0.0033586985043295723}
{'loss': 2.0757, 'grad_norm': 0.168923020362854, 'learning_rate': 1.388888888888889e-05, 'mean_token_accuracy': 0.5885903984308243, 'epoch': 0.004198373130411965}
{'loss': 2.0002, 'grad_norm': 0.1597493290901184, 'learning_rate': 1.6666666666666667e-05, 'mean_token_accuracy': 0.5985180847346783, 'epoch': 0.005038047756494359}
{'loss':

KeyboardInterrupt: 

In [40]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('llama-1.1B-rare_disease/tokenizer_config.json',
 'llama-1.1B-rare_disease/special_tokens_map.json',
 'llama-1.1B-rare_disease/tokenizer.model',
 'llama-1.1B-rare_disease/added_tokens.json',
 'llama-1.1B-rare_disease/tokenizer.json')

In [41]:
prompt = "is cystic fibrosis a rare diease?"
pipe = pipeline(task='text-generation', model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f'<s>[INST] {prompt} [/INST]')
print(result[0]['generated_text'])

<s>[INST] is cystic fibrosis a rare diease? [/INST] Cystic fibrosis is a rare disease. Cystic fibrosis (CF) is a genetic disease characterized by the presence of thick, sticky mucus in the lungs and other organs.  The disease is caused by mutations in the CFTR gene, which encodes a channel protein that regulates the flow of chloride ions across the cell membrane.  The mutations in the CFTR gene are responsible for the majority of CF cases.  The disease is inherited in an autosomal recessive manner.  The disease is characterized by the presence of thick, sticky mucus in the lungs and other organs, and is associated with a high mortality rate.  {1:Bailey, 2006}  {2:Bailey, 20
